# Rocket Launch Seismic/Infrasound Data Download and Response Correction

This notebook demonstrates how to download seismic and infrasound waveform data using ObsPy for all available stations within 200 miles of Kennedy Space Center during each rocket launch event provided in a CSV file. In addition, the notebook downloads instrument response information from the station inventory and removes the instrument response from the raw data.

## Workflow Overview

1. **CSV Input:** The notebook reads a CSV file (using pandas) that contains details for each launch event. The CSV is expected to have the following columns:
   - `name`: A unique event identifier
   - `launch_time`: Launch date/time in ISO 8601 format (e.g., `2025-01-15T14:30:00`)
   - `rocket_type`: The type of rocket (e.g., `Falcon 9`)
   - `mission`: A brief mission description
   - `payload`: Payload details
   - `pre_event`: *(Optional)* Seconds before launch for the start of the data window (default: 1800 seconds)
   - `post_event`: *(Optional)* Seconds after launch for the end of the data window (default: 7200 seconds)

2. **Station Inventory:** The notebook uses ObsPy's FDSN client (using IRIS as an example) to query the station inventory (including instrument response information) for stations within approximately 200 miles (321 km) of Kennedy Space Center.

3. **Waveform Download and Response Correction:** For each launch event, the notebook:
   - Downloads the raw waveform data for all channels in the inventory over a time window defined by the event's launch time and pre/post-event durations.
   - Saves the raw data as MiniSEED files.
   - Removes the instrument response (using the downloaded inventory) with example pre-filter parameters and outputs the corrected data (e.g., ground velocity) to new MiniSEED files.

Adjust the pre-filter parameters and output type based on your specific data and analysis requirements.

Let's begin!

In [30]:
import os
import pandas as pd
from obspy import UTCDateTime
from obspy.clients.fdsn import Client

# --- Configuration ---

# Kennedy Space Center approximate coordinates
KSC_LAT = 28.5729
KSC_LON = -80.6490

# 200 miles is roughly 321 km; convert km to degrees (1° ~ 111 km)
radius_km = 321
radius_deg = radius_km / 111.0  # ~2.89°

# CSV file containing launch event details
xls_file = '../../ksc_launch_events.xlsx'
#csv_file = 'ksc_launch_events_filtered.csv'
csv_file = 'merged_launches.csv'
launches_df = pd.read_csv(csv_file, index_col=None, parse_dates=['window_start', 'window_end', 'static_fire_time', 'spacex_launchtime'])
# Default pre_event and post_event durations (in seconds)
default_pre_event = 1800   # 30 minutes before launch
default_post_event = 7200  # 2 hours after launch

# Directory to store downloaded data
data_dir = os.path.join(os.path.expanduser('~'), 'data', "rocket_launch_data")
os.makedirs(data_dir, exist_ok=True)

# Initialize ObsPy FDSN client (using IRIS as the example provider)
client = Client("IRIS")

print("Configuration complete.")
display(launches_df)

Configuration complete.


,launch_designator,window_start,window_end,mission,SLC,launch_success,spacex_name,spacex_launchtime,static_fire_time,landing_attempt,landing_success,spacex_landingpad,spacex_landingtype
0,2010-004,2010-02-08 09:14:08+00:00,2010-02-08 09:14:08+00:00,Space Shuttle Endeavour / OV-105 | STS-130,39A,True,NaN,NaT,NaT,False,False,NaN,NaN
1,2010-012,2010-04-05 10:21:22+00:00,2010-04-05 10:21:22+00:00,Space Shuttle Discovery / OV-103 | STS-131,39A,True,NaN,NaT,NaT,False,False,NaN,NaN
2,2010-019,2010-05-14 18:20:09+00:00,2010-05-14 18:20:09+00:00,Space Shuttle Atlantis / OV-104 | STS-132,39A,True,NaN,NaT,NaT,False,False,NaN,NaN
3,2011-008,2011-02-24 21:53:24+00:00,2011-02-24 21:53:24+00:00,Space Shuttle Discovery / OV-103 | STS-133,39A,True,NaN,NaT,NaT,False,False,NaN,NaN
4,2011-020,2011-05-16 12:56:28+00:00,2011-05-16 12:56:28+00:00,Space Shuttle Endeavour / OV-105 | STS-134,39A,True,NaN,NaT,NaT,False,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,2025-024,2025-02-04 08:27:10+00:00,2025-02-04 11:24:19+00:00,Falcon 9 Block 5 | Starlink Group 12-3,40,True,NaN,NaT,NaT,False,False,NaN,NaN
400,2025-025,2025-02-04 23:07:00+00:00,2025-02-05 00:07:00+00:00,Falcon 9 Block 5 | WorldView Legion 5 & 6,39A,True,NaN,NaT,NaT,False,False,NaN,NaN
401,2025-027,2025-02-08 18:23:00+00:00,2025-02-08 22:08:00+00:00,Falcon 9 Block 5 | Starlink Group 12-9,40,True,NaN,NaT,NaT,False,False,NaN,NaN
402,2025-031,2025-02-11 18:53:20+00:00,2025-02-11 20:44:29+00:00,Falcon 9 Block 5 | Starlink Group 12-18,40,True,NaN,NaT,NaT,False,False,NaN,NaN


In [ ]:
'''
# --- Step 1: Read Launch Event Data from CSV using pandas ---
import json
import pandas as pd
print(os.getcwd())
if os.path.isfile(csv_file):
    launches_df = pd.read_csv(csv_file, index_col=None, parse_dates=['window_start', 'window_end'])
else:
    try:
        launches_df = pd.read_excel(xls_file, index_col=None, parse_dates=['window_start', 'window_end'])
        print(f"Successfully read {len(launches_df)} launch events from {csv_file}")
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        raise
    launches_df['SLC']=None
    launches_df['success']=True
    for i, row in launches_df.iterrows():
        pad_json = row['pad'].replace("'s ",'s ').replace("'", '"').replace('True','true').replace('False','false').replace('None','null')
        pad_dict = json.loads(pad_json)
        pad_name = pad_dict["name"]
        launches_df.iat[i, launches_df.columns.get_loc('SLC')] = pad_name
        if 'Failure' in row['status']:
            launches_df.iat[i, launches_df.columns.get_loc('status')] = 'Failure'
    launches_df = launches_df[['launch_designator', 'window_start', 'window_end', 'name', 'SLC', 'success'] ]
    launches_df.sort_values("window_start", inplace=True)
    launches_df.to_csv(csv_file, index=False)
launches_df.head(100)
'''

/Volumes/ExtremeSSD1TB/Developer/GitHub/KSCRocketSeismoHydrology/Python/new_workflow


,launch_designator,window_start,window_end,name,SLC,success
0,2010-004,2010-02-08 09:14:08+00:00,2010-02-08 09:14:08+00:00,Space Shuttle Endeavour / OV-105 | STS-130,Launch Complex 39A,True
1,2010-012,2010-04-05 10:21:22+00:00,2010-04-05 10:21:22+00:00,Space Shuttle Discovery / OV-103 | STS-131,Launch Complex 39A,True
2,2010-019,2010-05-14 18:20:09+00:00,2010-05-14 18:20:09+00:00,Space Shuttle Atlantis / OV-104 | STS-132,Launch Complex 39A,True
3,2011-008,2011-02-24 21:53:24+00:00,2011-02-24 21:53:24+00:00,Space Shuttle Discovery / OV-103 | STS-133,Launch Complex 39A,True
4,2011-020,2011-05-16 12:56:28+00:00,2011-05-16 12:56:28+00:00,Space Shuttle Endeavour / OV-105 | STS-134,Launch Complex 39A,True
...,...,...,...,...,...,...
95,2020-048,2020-07-20 21:00:00+00:00,2020-07-21 00:55:00+00:00,Falcon 9 Block 5 | ANASIS-II,Space Launch Complex 40,True
96,2020-052,2020-07-30 11:50:00+00:00,2020-07-30 13:50:00+00:00,Atlas V 541 | Mars 2020 (Perseverance rover & ...,Space Launch Complex 41,True
97,2020-055,2020-08-07 05:12:00+00:00,2020-08-07 05:12:00+00:00,Falcon 9 Block 5 | Starlink 9,Launch Complex 39A,True
98,2020-057,2020-08-18 14:31:00+00:00,2020-08-18 14:31:00+00:00,Falcon 9 Block 5 | Starlink 10,Space Launch Complex 40,True


In [34]:
counts = launches_df["SLC"].value_counts()
print(counts)
counts = launches_df["spacex_landingpad"].value_counts()
print(counts)

df_subset = launches_df[launches_df['spacex_landingpad'].str.contains('LZ', na=False)]
display(df_subset)

SLC
40                                 223
39A                                120
41                                  38
37B                                  8
46                                   4
Rocket Lab  1B                       3
Cape Canaveral                       2
37A                                  1
Rocket Lab  1A                       1
39B                                  1
Rocket Lab  2 (Launch Area 0 C)      1
16                                   1
36A                                  1
Name: count, dtype: int64
spacex_landingpad
LZ-1    19
JRTI    15
ASOG     6
LZ-2     3
Name: count, dtype: int64


,launch_designator,window_start,window_end,mission,SLC,launch_success,spacex_name,spacex_launchtime,static_fire_time,landing_attempt,landing_success,spacex_landingpad,spacex_landingtype
15,2016-046,2016-07-18 04:45:29+00:00,2016-07-18 04:45:29+00:00,Falcon 9 Full Thrust | SpX CRS-9,40,True,CRS-9,2016-07-18 04:45:00+00:00,2016-07-16 02:31:47+00:00,True,True,LZ-1,RTLS
27,2017-009,2017-02-19 14:39:00+00:00,2017-02-19 14:39:00+00:00,Falcon 9 Full Thrust | SpX CRS-10,39A,True,CRS-10,2017-02-19 14:39:00+00:00,2017-02-12 21:30:00+00:00,True,True,LZ-1,RTLS
32,2017-022,2017-05-01 11:00:00+00:00,2017-05-01 13:00:00+00:00,Falcon 9 Full Thrust | NROL-76,39A,True,NROL-76,2017-05-01 11:15:00+00:00,2017-04-25 19:02:00+00:00,True,True,LZ-1,RTLS
35,2017-030,2017-06-03 21:07:38+00:00,2017-06-03 21:07:38+00:00,Falcon 9 Full Thrust | SpX CRS-11,39A,True,CRS-11,2017-06-03 21:07:00+00:00,2017-05-28 16:00:00+00:00,True,True,LZ-1,RTLS
39,2017-045,2017-08-14 16:31:37+00:00,2017-08-14 16:31:37+00:00,Falcon 9 Block 4 | SpX CRS-12,39A,True,CRS-12,2017-08-14 16:31:00+00:00,2017-08-10 13:10:00+00:00,True,True,LZ-1,RTLS
42,2017-052,2017-09-07 13:50:00+00:00,2017-09-07 18:55:00+00:00,Falcon 9 Block 4 | OTV-5 (X-37B),39A,True,Boeing X-37B OTV-5,2017-09-07 13:50:00+00:00,2017-08-31 20:30:00+00:00,True,True,LZ-1,RTLS
47,2017-080,2017-12-15 15:36:09+00:00,2017-12-15 15:36:09+00:00,Falcon 9 Full Thrust | SpX CRS-13,40,True,CRS-13,2017-12-15 15:36:00+00:00,2017-12-06 20:00:00+00:00,True,True,LZ-1,RTLS
48,2018-001,2018-01-08 01:00:00+00:00,2018-01-08 03:00:00+00:00,Falcon 9 Block 4 | Zuma,40,True,ZUMA,2018-01-08 01:00:00+00:00,2017-11-11 23:00:00+00:00,True,True,LZ-1,RTLS
52,2018-017,2018-02-06 18:30:00+00:00,2018-02-06 21:00:00+00:00,Falcon Heavy | Demo (Test Flight),39A,True,Falcon Heavy Test Flight,2018-02-06 20:45:00+00:00,2018-01-24 17:30:00+00:00,True,True,LZ-2,RTLS
53,2018-017,2018-02-06 18:30:00+00:00,2018-02-06 21:00:00+00:00,Falcon Heavy | Demo (Test Flight),39A,True,Falcon Heavy Test Flight,2018-02-06 20:45:00+00:00,2018-01-24 17:30:00+00:00,True,True,LZ-1,RTLS


In [ ]:
# --- Step 2: Query Station Inventory (including instrument response) ---

print("Querying station inventory within 200 miles of Kennedy Space Center...")
try:
    inventory = client.get_stations(latitude=KSC_LAT, longitude=KSC_LON,
                                    maxradius=radius_deg, level="channel")
    # Optionally, save inventory to file for inspection
    inventory.write("stations.xml", format="STATIONXML")
    network_count = len(inventory.get_contents().get("networks", []))
    print(f"Found {network_count} networks.")
except Exception as e:
    print("Error retrieving station inventory:", e)
    raise

In [ ]:
# --- Step 3: Process Each Rocket Launch Event ---

for idx, row in launches_df.iterrows():
    launch_name = row["name"]
    launch_time_str = row["launch_time"]

    try:
        launch_time = UTCDateTime(launch_time_str)
    except Exception as e:
        print(f"Error parsing launch time for {launch_name}: {e}")
        continue

    # Use provided pre_event and post_event if available; otherwise, defaults.
    pre_event = int(row["pre_event"]) if "pre_event" in row and not pd.isna(row["pre_event"]) else default_pre_event
    post_event = int(row["post_event"]) if "post_event" in row and not pd.isna(row["post_event"]) else default_post_event

    start_time = launch_time - pre_event
    end_time = launch_time + post_event

    print(f"\nProcessing event: {launch_name}")
    print(f"  Launch time: {launch_time}")
    print(f"  Time window: {start_time} to {end_time}")
    print(f"  Rocket Type: {row.get('rocket_type', '')}")
    print(f"  Mission: {row.get('mission', '')}")
    print(f"  Payload: {row.get('payload', '')}")

    # Create a directory for this launch event
    event_dir = os.path.join(data_dir, launch_name)
    os.makedirs(event_dir, exist_ok=True)

    # Loop over each network, station, and channel in the inventory to download waveforms
    for network in inventory:
        for station in network:
            station_code = station.code
            for channel in station:
                channel_code = channel.code
                # Some channels may have an empty location code; use empty string if so.
                location_code = channel.location_code if channel.location_code else ""
                try:
                    # Request waveform data for the specified time window
                    st = client.get_waveforms(network.code, station_code, location_code, channel_code,
                                              start_time, end_time)
                    
                    # Save the raw waveform data as MiniSEED
                    raw_filename = f"{network.code}.{station_code}.{location_code}.{channel_code}.{launch_name}.mseed"
                    raw_filepath = os.path.join(event_dir, raw_filename)
                    st.write(raw_filepath, format="MSEED")
                    print(f"  Downloaded raw data: {network.code}.{station_code}.{channel_code}")
                    
                    # --- Instrument Response Removal ---
                    try:
                        # Detrend the data
                        st.detrend(type="linear")
                        st.detrend(type="demean")
                        
                        # Remove instrument response
                        # Adjust the pre_filt parameters as needed
                        st.remove_response(inventory=inventory,
                                           output="VEL",   # Change to "DISP" for displacement if needed
                                           pre_filt=(0.01, 0.02, 8.0, 10.0),
                                           water_level=60)
                        
                        corrected_filename = raw_filename.replace(".mseed", ".corr.mseed")
                        corrected_filepath = os.path.join(event_dir, corrected_filename)
                        st.write(corrected_filepath, format="MSEED")
                        print(f"  Instrument response removed: {network.code}.{station_code}.{channel_code}")
                    except Exception as e:
                        print(f"  Failed to remove instrument response for {network.code}.{station_code}.{channel_code} -- {e}")

                except Exception as e:
                    print(f"  Failed to download data for {network.code}.{station_code}.{channel_code} -- {e}")

print("\nData download and instrument response correction complete.")